## hand representation
- bit-vector

In [4]:
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import interact
display(HTML("<style>.container { width:75% !important; }</style>"))

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## Params

In [6]:
_RUN_UNIT_TESTS = True

## Game constants

In [170]:
# game constants
NUM_RANKS = 13
NUM_SUITS = 4
NUM_CARDS = NUM_RANKS * NUM_SUITS
CARDS = [(s, r) for s in range(NUM_SUITS) for r in range(NUM_RANKS)]

STRAIGHT_FLUSH_LEN = 5
STRAIGHT_LEN = 5
FLUSH_LEN = 5
FOUR_KIND_LEN = 4
THREE_KIND_LEN = 3
TWO_KIND_LEN = 2


In [227]:
# ratings
RATING_OFFSET = 2 * NUM_RANKS

STRAIGHT_FLUSH_RATING = 9 << RATING_OFFSET
FOUR_KIND_RATING = 8 << RATING_OFFSET
FULL_HOUSE_RATING = 7 << RATING_OFFSET
FLUSH_RATING = 6 << RATING_OFFSET
STRAIGHT_RATING = 5 << RATING_OFFSET
THREE_KIND_RATING = 4 << RATING_OFFSET
TWO_PAIR_RATING = 3 << RATING_OFFSET
TWO_KIND_RATING = 2 << RATING_OFFSET
ONE_KIND_RATING = 1 << RATING_OFFSET


RATING_MASK = 15 << RATING_OFFSET


## Masks
- _suit_mask: _suit_mask[s] gives bit mask for all cards of suit s
- _bit_mask: _bit_mask[ind] gives bit mask for bitind 'ind'
- _bit_mask2: _bit_mask2[s][r] gives bit mask for card (s, r)

In [289]:
_all_mask = (1 << NUM_CARDS) - 1
_bit_mask = [1 << i for i in range(NUM_CARDS)]
_bit_mask2 = [[(1 << (s * NUM_RANKS + r)) for r in range(NUM_RANKS)] for s in range(NUM_SUITS)]

In [292]:
_suit_masks = [((1 << NUM_RANKS) - 1) << (s * NUM_RANKS) for s in range(NUM_SUITS)]
_rank_masks = [sum([1 << (s * NUM_RANKS + r) for s in range(NUM_SUITS)]) for r in range(NUM_RANKS)]

_suit_mask = _suit_masks[0]

In [293]:
base_masks = []
for s1 in range(NUM_SUITS):
    for s2 in range(s1):
        val = _bit_mask[s1 * NUM_RANKS] + _bit_mask[s2 * NUM_RANKS]
        base_masks += [val]

_2k_masks = []
for r in range(NUM_RANKS):
    _2k_masks += [bmask << r for bmask in base_masks]

## Utils

In [294]:
def binify(bits_list):
    return [bin(b) for b in bits_list]

In [295]:
def run_unit_tests(unit_fn):
    unit_fn()
    print(f"`{unit_fn.__name__}` success!")

In [286]:
def get_bits_by_suit(bits):
    vals = []
    for s in range(NUM_SUITS):
        vals += [(bits >> (s * NUM_RANKS)) & _suit_mask]
    return vals

In [297]:
def inv_bits(bits):
    return bits ^ _all_mask

In [235]:
# returns bit vector of NUM_RANKS length
def get_kicker_bits(bits):
    total = 0
    for s in range(NUM_SUITS):
        total = total | (bits >> (s * NUM_RANKS))
    
    return total & _suit_mask

def get_2k_bits(bits):
    bits_by_suit = get_bits_by_suit(bits)
    
    
    

In [232]:
# returns highest bit (1 << bitind)

def get_n_highest_bits(bits, n):
    v = 0
    for i in range(n):
        b = get_highest_bit(bits)
        bits -= b
        v += b
    return v

def get_highest_bit(bits):
    # ensure it has at least one bit
    assert bits > 0    
    return _bit_mask[bits.bit_length() - 1]



In [231]:
## cards are a tuple representation of a hand (bit-vector)
def card_to_ind(s, r):
    assert 0 <= s < NUM_SUITS and 0 <= r < NUM_RANKS
    return s * NUM_RANKS + r

def ind_to_card(ind):
    assert 0 <= ind < NUM_CARDS
    return (ind // NUM_RANKS, ind % NUM_RANKS)

def hand_to_cards(hand):
    assert 0 <= hand < (1 << NUM_CARDS)
    
    cards = []
    for ind, card in enumerate(CARDS):
        if (hand & _bit_mask[ind]) > 0:
            cards += [card]
            
    return cards

def cards_to_hand(cards):    
    hand = 0
    for s, r in cards:
        hand += _bit_mask[card_to_ind(s, r)]
    return hand

            
def unit_tests__card_to_ind():
    assert ( card_to_ind(0, 0) == 0 )
    assert ( card_to_ind(0, 1) == 1 )
    assert ( card_to_ind(1, 1) == 14 )
    assert ( card_to_ind(2, 6) == 32 )
    assert ( card_to_ind(3, 12) == 51 )
    
def unit_tests__ind_to_card():
    assert ( ind_to_card(0) == (0, 0) )
    assert ( ind_to_card(5) == (0, 5) )
    assert ( ind_to_card(13) == (1, 0) )
    assert ( ind_to_card(51) == (3, 12) )
    

def unit_tests__hand_to_cards():
    assert ( tuple(hand_to_cards(0b11)) == ((0, 0), (0, 1)) )
    assert ( tuple(hand_to_cards((1 << 13) + 7)) == ((0, 0), (0, 1), (0, 2), (1, 0)) )
    assert ( tuple(hand_to_cards((3 << 13) + 7)) == ((0, 0), (0, 1), (0, 2), (1, 0), (1, 1)) )


if _RUN_UNIT_TESTS:
    run_unit_tests(unit_tests__card_to_ind)
    run_unit_tests(unit_tests__ind_to_card)
    run_unit_tests(unit_tests__hand_to_cards)
    

`unit_tests__card_to_ind` success!
`unit_tests__ind_to_card` success!
`unit_tests__hand_to_cards` success!


## ratings

- straight flush (SF)
    - highest value (1 number, 13 bits)

- four of a kind (4K)
    - four + kicker (1 +  numbers, 13 bits)

- full house (FH)
    - triple + pair (1 + 1 numbers, 26 bits)

- flush (FL)
    - highest five numbers (5 numbers, 13 bits)
  
- straght (ST)
    - highest value 1 number, 13 bits)

- triple (3K)
    - triple + kicker1 + kicker2 (1 + 2 highest numbers, 26 bits)
    
- two pair (2P)
    - pair1 + pair2 + kicker (2 + 1 highest number, 26 bits)

- pair (2K)
    - pair + kicker1 + kicker2 + kicker3 (1 + 3 highest numbers, 13 bits)

- high card (1K)
    - highest five numbers (5 nums, 13 bits)

In [233]:
# check for a straight flush
base_mask = (1 << STRAIGHT_FLUSH_LEN) - 1
_sf_masks = [base_mask << (r + s * NUM_RANKS) for r in range(NUM_RANKS - STRAIGHT_FLUSH_LEN + 1) for s in range(NUM_SUITS)]


In [234]:
def check_straight_flush(n):
    is_sf_mask = (1 << STRAIGHT_FLUSH_LEN) - 1

    rating = 0

    # since this goes from small to big, we can use `rating = val`
    for mask in _sf_masks:
        masked = mask & n
        
        ind = (mask.bit_length() - 1) // NUM_RANKS * NUM_RANKS
        
        if masked.bit_count() == STRAIGHT_FLUSH_LEN:
            rating = masked >> ind

    if rating > 0:
        return STRAIGHT_FLUSH_RATING + rating
    else:
        return 0
    
def unit_tests__check_straight_flush():
    assert check_straight_flush(0b11111) == STRAIGHT_FLUSH_RATING + 0b11111
    assert check_straight_flush(0b11111 << 13) == STRAIGHT_FLUSH_RATING + 0b11111
    assert check_straight_flush(0b11111 << 12) == 0
    assert check_straight_flush(0b11111 << 9) == 0
    assert check_straight_flush(0b11111 << 8) == STRAIGHT_FLUSH_RATING + (0b11111 << 8)
    assert check_straight_flush(0b11111 << 39) == STRAIGHT_FLUSH_RATING + (0b11111)
    assert check_straight_flush(0b11111 << 47) == STRAIGHT_FLUSH_RATING + (0b11111 << 8)
    
    assert check_straight_flush(0b1111111) == STRAIGHT_FLUSH_RATING + (0b11111 << 2)
    assert check_straight_flush(0b111111 << 8) == STRAIGHT_FLUSH_RATING + (0b11111 << 8)
    assert check_straight_flush(0b11111 << 12) == 0
    assert check_straight_flush(0b111111 << 12) == STRAIGHT_FLUSH_RATING + (0b11111)
    

    
run_unit_tests(unit_tests__check_straight_flush)

`unit_tests__check_straight_flush` success!


In [237]:
_four_kind_masks = _rank_masks


In [279]:
def check_four_kind(bits):

    rating = 0

    for mask in _four_kind_masks:
        masked = bits & mask
        if masked.bit_count() == FOUR_KIND_LEN:
            rating = masked & _suit_mask

    if rating > 0:

        # find kicker

        kbits = get_kicker_bits(bits) - rating
        kicker = get_highest_bit(kbits)

        return FOUR_KIND_RATING + (rating << NUM_RANKS) + kicker
    else:
        return 0
    
def unit_tests__check_four_kind():

    base_bits = (1 << 0) + (1 << 13) + (1 << 26) + (1 << 39)

    bits = base_bits + (base_bits << 10) + (1 << 3)
    ans = FOUR_KIND_RATING + (1 << 10 << NUM_RANKS) + (1 << 3)
    assert check_four_kind(bits) == ans
    
    bits = (base_bits - 1)
    ans = 0
    assert check_four_kind(bits) == ans

    bits = (base_bits << 5) + (base_bits << 10) + (1 << 3)
    ans = FOUR_KIND_RATING + (1 << 10 << NUM_RANKS) + (1 << 5)
    assert check_four_kind(bits) == ans
    
    bits = (1 << NUM_CARDS) - 1
    ans = FOUR_KIND_RATING + (1 << (NUM_RANKS - 1) << NUM_RANKS) + (1 << (NUM_RANKS - 2))
    assert check_four_kind(bits) == ans
    
    bits = (1 << NUM_CARDS) - 1
    ans = FOUR_KIND_RATING + (1 << (NUM_RANKS - 1) << NUM_RANKS) + (1 << (NUM_RANKS - 2))
    assert check_four_kind(bits) == ans
    
run_unit_tests(unit_tests__check_four_kind)

here
`unit_tests__check_four_kind` success!


In [268]:
kbits

9

In [265]:
bin(rating)

'0b10000000000'

In [200]:
([b for b in _sf_masks][-4] & 0b1111100000000).bit_count()

5